# Features Seba

La idea de esta notebook es crear los siguientes features:

 - [screen_resolution](#screen_resolution)
 - [cantidad de eventos por categoría](#cantidad-de-eventos-por-categoría)
 - [cantidad de eventos](#cantidad-de-eventos)
 - [fechas](#fechas)
 - [precios](#precios)
     - [precios por modelo](#precios-por-modelo)
     - [precios por fecha](#precios-por-fecha)
 - [diferencia de precios](#diferencia-de-precios) 
     - [diferencia de precios por modelo](#diferencia-de-precios-por-modelo)
     - [diferencia de precios por fecha](#diferencia-de-precios-por-fecha)

In [1]:
from sklearn.feature_extraction import FeatureHasher
import requests, json, os, re

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

limpieza_path = '"{}"'.format(os.path.join(current_folder, '..', 'Limpieza.ipynb'))

In [3]:
modelos_a_buscar_path = os.path.join(current_folder, '..', 'modelos_a_buscar.csv')
precios_a_buscar_path = os.path.join(current_folder, '..', 'precios_a_buscar.csv')
precios_resultado_path = os.path.join(current_folder, '..', 'precios_resultado.csv')

In [4]:
%run $limpieza_path
df = get_clean_df()

In [5]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


#### screen_resolution

Para este atributos se utilizó el promedio entre los distintos valores, sin tener en cuenta los nulos.

In [6]:
df_features['screen_resolution_height'] = df.groupby('person')['screen_resolution_height'].mean()
df_features['screen_resolution_width'] = df.groupby('person')['screen_resolution_width'].mean()

#### cantidad de eventos por categoría

La idea de este feature es tener una columna por categoría que indique la cantidad de eventos realizados por usuario en dicha categoría.

In [7]:
events_per_category_count = df.groupby('person')['event'].value_counts()
events_per_category_count.head()

person    event          
0008ed71  checkout             3
          visited site         2
          generic listing      1
00091926  viewed product     372
          visited site        34
Name: event, dtype: int64

In [8]:
events_per_category = events_per_category_count.unstack().fillna(0)
events_per_category.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0


In [9]:
df_features = df_features.merge(events_per_category, how='inner', on='person')

Chequeo que no haya habido claves que se hayan quedado fuera del merge:

In [10]:
df_features.shape[0] == df['person'].unique().shape[0]

True

#### cantidad de eventos

In [11]:
df_features['event_count'] = df.groupby('person')['event'].count()

In [12]:
df_features.head()

,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,event_count
person,,,,,,,,,,,,,,
4886f805,640.0,360.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0,9
ad93850f,640.0,360.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0,65
0297fc1e,640.0,360.0,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0,567
2d681dd8,768.0,1360.0,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0,26
cccea85e,864.0,1536.0,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0,836


#### regiones

In [13]:
df_region_count = df.groupby('person')['region'].value_counts().unstack().fillna(0)

In [14]:
df_region_count.head()

region,Acre,Alabama,Alagoas,Amapa,Amazonas,Arizona,Arkansas,Astrakhanskaya Oblast',Asuncion,Aveiro,Bahia,British Columbia,...,Tinh Thua Thien-Hue,Tocantins,Tokyo,Tul'skaya Oblast',Turin,Tyrol,Utah,Virginia,Washington,Western Cape,Wisconsin,Wrexham
person,,,,,,,,,,,,,,,,,,,,,,,,,
00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_features = df_features.merge(df_region_count, how='left', on='person')

In [16]:
df_features[df_region_count.columns].fillna(0, inplace=True)

/home/seba/py35/lib/python3.5/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [17]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

#### fechas

Calculo el promedio y desviación estándar de días hasta el 31-05

In [18]:
df['dias hasta el 31-05'] = (df['timestamp'].max() - df['timestamp']).dt.days

In [19]:
days_mean_std = df.groupby('person')['dias hasta el 31-05'].agg(['mean', 'std'])
days_mean_std.rename(columns={'mean':'days until 31-05 mean', 'std':'days until 31-05 std'}, inplace=True)

In [20]:
df_features = df_features.merge(days_mean_std, how='inner', on='person')

In [21]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

#### precios

A través de Web scraping se obtuvieron los precios de los distintos modelos de trocafone al día 16/11/18 y se guardaron el archivo "precios.csv".

Para ello primero se guardó en el csv los modelos que había que buscar.

In [22]:
precios_a_buscar = df.groupby(['sku', 'model', 'condition', 'storage_gb'])['color'].first()\
                    .reset_index(name='color').set_index('sku')
precios_a_buscar.head()

,model,condition,storage_gb,color
sku,,,,
71.0,iPhone 4G,Excelente,8.0,Negro
74.0,iPhone 4G,Muy bueno,8.0,Negro
80.0,iPhone 4G,Muy bueno,8.0,Blanco
81.0,iPhone 4G,Excelente,8.0,Blanco
87.0,iPhone 4G,Muy bueno,16.0,Negro


In [23]:
pd.DataFrame(precios_a_buscar['model'].unique(), columns=['model']).set_index('model').to_csv(modelos_a_buscar_path)

Luego se realizó el scraping y el resultado fue el siguiente:

In [24]:
precios = pd.read_csv(precios_resultado_path, index_col='sku')
precios.head()

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
71.0,iPhone 4G,1.0,Preto,15.0,8GB,71.0,8.0,5.0,used,False,iPhone 4G 8GB Preto,Excelente,...,False,54.0,0.0,True,12.0,26.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#8-iphone-4g-8gb-preto-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
74.0,iPhone 4G,1.0,Preto,15.0,8GB,74.0,8.0,6.0,used,False,iPhone 4G 8GB Preto,Muito Bom,...,False,58.0,1.0,True,12.0,24.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#8-iphone-4g-8gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
80.0,iPhone 4G,2.0,Branco,15.0,8GB,80.0,9.0,6.0,used,False,iPhone 4G 8GB Branco,Muito Bom,...,False,58.0,0.0,True,12.0,24.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#9-iphone-4g-8gb-branco-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
81.0,iPhone 4G,2.0,Branco,15.0,8GB,81.0,9.0,5.0,used,False,iPhone 4G 8GB Branco,Excelente,...,False,54.0,0.0,True,12.0,26.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#9-iphone-4g-8gb-branco-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 8GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
87.0,iPhone 4G,1.0,Preto,16.0,16GB,87.0,10.0,6.0,used,False,iPhone 4G 16GB Preto,Muito Bom,...,False,54.0,1.0,True,12.0,28.25,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#10-iphone-4g-16gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 4G 16GB Preto em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."


Chequeamos que todos los modelos tengan datos.

In [25]:
precios[precios['price_current'].isna()].shape[0] == 0

False

Vemos sobre cuales no encontramos resultados.

In [26]:
no_encontrados = precios[precios['price_current'].isna()]['model'].unique()
no_encontrados

array(['iPhone 5', 'iPhone 6S Plus', 'Samsung Galaxy Note 8',
       'Samsung Galaxy A7 2017'], dtype=object)

In [27]:
precios[precios['price_current'].isna()].index.unique()

Float64Index([5925.0, 5926.0, 7088.0, 14296.0, 14734.0], dtype='float64', name='sku')

Sobre estos 5 sku's no tenemos datos, pero sí sobre otros sku's que corresponden a los mismos modelos.

In [28]:
precios.loc[precios['model'].isin(no_encontrados)].head()

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
171.0,iPhone 5,1.0,Preto,16.0,16GB,171.0,22.0,6.0,used,False,iPhone 5 16GB Preto,Muito Bom,...,False,50.0,0.0,True,12.0,49.92,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#22-iphone-5-16gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
172.0,iPhone 5,1.0,Preto,16.0,16GB,172.0,22.0,5.0,used,False,iPhone 5 16GB Preto,Excelente,...,False,45.0,0.0,True,12.0,54.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#22-iphone-5-16gb-preto-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."
178.0,iPhone 5,2.0,Branco,16.0,16GB,178.0,23.0,6.0,used,False,iPhone 5 16GB Branco,Muito Bom,...,False,50.0,1.0,True,12.0,49.92,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#23-iphone-5-16gb-branco-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
179.0,iPhone 5,2.0,Branco,16.0,16GB,179.0,23.0,5.0,used,False,iPhone 5 16GB Branco,Excelente,...,False,45.0,0.0,True,12.0,54.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#23-iphone-5-16gb-branco-excelente-5,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."
185.0,iPhone 5,1.0,Preto,17.0,32GB,185.0,24.0,6.0,used,False,iPhone 5 32GB Preto,Muito Bom,...,False,47.0,1.0,True,12.0,61.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#24-iphone-5-32gb-preto-muito-bom-6,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 32GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."


Por ello vamos a buscar las características de estos sku's y completaremos con los que son parecidos (del mismo modelo).

Para los **iPhone 5**.

In [29]:
iphone5_skus = precios[precios['price_current'].isna() & (precios['model']=='iPhone 5')].index
iphone5_skus

Float64Index([5925.0, 5926.0], dtype='float64', name='sku')

In [30]:
df[df['sku']==5925].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
20820,iPhone 5,Bueno - Sin Touch ID,16.0,Blanco


In [31]:
df[df['sku']==5926].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
16431,iPhone 5,Bueno - Sin Touch ID,16.0,Negro


Por lo tanto vamos a elegir los modelos que más se parezcan a estos.

In [32]:
precios[(precios['model']=='iPhone 5') & (precios['condition_name']=='Bom')\
        & (precios['color_name']=='Branco') & (precios['storage_name']=='16GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
2710.0,iPhone 5,2.0,Branco,16.0,16GB,2710.0,23.0,15.0,used,False,iPhone 5 16GB Branco,Bom,...,False,55.0,0.0,True,12.0,44.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#23-iphone-5-16gb-branco-bom-15,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Branco em oferta | Troca...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [33]:
precios.loc[5925] = precios.loc[2710]

In [34]:
precios[(precios['model']=='iPhone 5') & (precios['condition_name']=='Bom')\
        & (precios['color_name']=='Preto') & (precios['storage_name']=='16GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
2711.0,iPhone 5,1.0,Preto,16.0,16GB,2711.0,22.0,15.0,used,False,iPhone 5 16GB Preto,Bom,...,False,55.0,1.0,True,12.0,44.08,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#22-iphone-5-16gb-preto-bom-15,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 5 16GB Preto em oferta | Trocafone,"[{""original"": ""https://imagens.trocafone.com/i..."


In [35]:
precios.loc[5926] = precios.loc[2711]

Para el **iPhone 6S Plus**.

In [36]:
iphone6_sku = precios[precios['price_current'].isna() & (precios['model']=='iPhone 6S Plus')].index
iphone6_sku

Float64Index([7088.0], dtype='float64', name='sku')

In [37]:
df[df['sku']==iphone6_sku].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
37798,iPhone 6S Plus,Bueno - Sin Touch ID,64.0,Gris Espacial


In [38]:
precios[(precios['model']=='iPhone 6S Plus') & (precios['condition_name']=='Bom')\
        & (precios['color_name']=='Cinza espacial') & (precios['storage_name']=='64GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
7085.0,iPhone 6S Plus,9.0,Cinza espacial,18.0,64GB,7085.0,497.0,15.0,used,False,iPhone 6S Plus 64GB Cinza Espacial,Bom,...,False,48.0,1.0,True,12.0,161.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#497-iphone-6s-plus-64gb-cinza-espacial-bom-15,https://www.trocafone.com/celulares/iphone/m/i...,Comprar iPhone 6S Plus 64GB Cinza Espacial em ...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [39]:
precios.loc[iphone6_sku[0]] = precios.loc[7085]

Para el **Samsung Galaxy Note 8**.

In [40]:
note8_sku = precios[precios['price_current'].isna() & (precios['model']=='Samsung Galaxy Note 8')].index
note8_sku

Float64Index([14296.0], dtype='float64', name='sku')

In [41]:
df[df['sku']==note8_sku].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
73,Samsung Galaxy Note 8,Nuevo,64.0,Negro


In [42]:
precios[(precios['model']=='Samsung Galaxy Note 8')]['condition_name'].unique()

array(['Bom', 'Muito Bom', 'Excelente', nan], dtype=object)

En este caso nos llevamos la sorpresa de no encontrar ningún celular con estado *nuevo*, por lo cual se complica la tarea de completar los datos.

In [43]:
precios[(precios['model']=='Samsung Galaxy Note 8') & (precios['condition_name']=='Excelente')\
        & (precios['color_name']=='Preto') & (precios['storage_name']=='64GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
13180.0,Samsung Galaxy Note 8,1.0,Preto,18.0,64GB,13180.0,936.0,5.0,used,False,Samsung Galaxy Note 8 64GB Preto,Excelente,...,False,54.0,0.0,True,12.0,166.58,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#936-samsung-galaxy-note-8-64gb-preto-excelente-5,https://www.trocafone.com/celulares/samsung/ga...,Comprar Samsung Galaxy Note 8 64GB Preto em of...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [44]:
precios.loc[note8_sku[0]] = precios.loc[13180]

Pero vamos a modificar ciertos atributos.

In [45]:
precios.at[note8_sku[0], 'cellphone_condition_type'] = 'new'
precios.at[note8_sku[0], 'condition_name'] = 'Novo'
precios.at[note8_sku[0], 'price_current'] = precios.at[note8_sku[0], 'price_new']
precios.at[note8_sku[0], 'price_invoice_current'] = precios.at[note8_sku[0], 'price_new']
precios.at[note8_sku[0], 'price_one_installment_current'] = precios.at[note8_sku[0], 'price_new']
precios.at[note8_sku[0], 'stock'] = 0

Para el **Samsung Galaxy A7 2017**.

In [46]:
a7_sku = precios[precios['price_current'].isna() & (precios['model']=='Samsung Galaxy A7 2017')].index
a7_sku

Float64Index([14734.0], dtype='float64', name='sku')

In [47]:
df[df['sku']==a7_sku].head(1)[['model', 'condition', 'storage_gb', 'color']]

,model,condition,storage_gb,color
1041,Samsung Galaxy A7 2017,Nuevo,32.0,Negro


In [48]:
precios[(precios['model']=='Samsung Galaxy A7 2017')]['condition_name'].unique()

array(['Bom', 'Muito Bom', 'Excelente', nan], dtype=object)

Nuevamente no encontramos ningún celular con estado *nuevo*, y hacemos lo mismo que antes.

In [49]:
precios[(precios['model']=='Samsung Galaxy A7 2017') & (precios['condition_name']=='Excelente')\
        & (precios['color_name']=='Preto') & (precios['storage_name']=='32GB')]

,model,color_id,color_name,storage_id,storage_name,trocable_id,product_id,cellphone_condition_id,cellphone_condition_type,enable_membership,product_name,condition_name,...,featured,discount,stock,super_discount,installments,installment_value,interest,installments_info,urlHash,url,title,images
sku,,,,,,,,,,,,,,,,,,,,,,,,,
10884.0,Samsung Galaxy A7 2017,1.0,Preto,17.0,32GB,10884.0,772.0,5.0,used,False,Samsung Galaxy A7 2017 Preto,Excelente,...,False,55.0,1.0,True,12.0,80.75,0.0,"{""gatewayMethod"": ""min"", ""installments"": [{""am...",#772-samsung-galaxy-a7-2017-preto-excelente-5,https://www.trocafone.com/celulares/samsung/ga...,Comprar Samsung Galaxy A7 2017 Preto em oferta...,"[{""original"": ""https://imagens.trocafone.com/i..."


In [50]:
precios.loc[a7_sku[0]] = precios.loc[10884]

Pero vamos a modificar ciertos atributos.

In [51]:
precios.at[a7_sku[0], 'cellphone_condition_type'] = 'new'
precios.at[a7_sku[0], 'condition_name'] = 'Novo'
precios.at[a7_sku[0], 'price_current'] = precios.at[a7_sku[0], 'price_new']
precios.at[a7_sku[0], 'price_invoice_current'] = precios.at[a7_sku[0], 'price_new']
precios.at[a7_sku[0], 'price_one_installment_current'] = precios.at[a7_sku[0], 'price_new']
precios.at[a7_sku[0], 'stock'] = 0

Volvemos a chequear que todos los modelos tengan datos.

In [52]:
assert(precios[precios['price_current'].isna()].shape[0] == 0)

Convierto los sku's de los precios en categorías para poder hacer el merge con el dataframe

In [53]:
precios.index = precios.index.astype('category')

Hacemos el merge, y llenamos los nan para que sean incluidos en el groupby. (Esta solución deberíamos mejorarla de alguna forma). 

### precios por modelo

In [54]:
columna_f_h = 'model_x'

In [55]:
df_merged = df.merge(precios, how='left', on='sku')
df_merged[columna_f_h].cat.add_categories(['nan'], inplace=True)
df_merged[columna_f_h].fillna('nan', inplace=True)

In [56]:
grouped = df_merged.groupby(['person', columna_f_h]).agg({'price_current': 'mean'}).unstack()
grouped.columns = grouped.columns.droplevel()
assert(grouped.shape[0] == df_features.shape[0])
grouped.head()

model_x,Asus Live,Asus Zenfone 2,Asus Zenfone 2 Deluxe,Asus Zenfone 2 Laser,"Asus Zenfone 2 Laser 6""",Asus Zenfone 3 Max 32 GB,Asus Zenfone 3 Max 16 GB,Asus Zenfone 5,Asus Zenfone 6,Asus Zenfone Go,Asus Zenfone Selfie,LG X Screen,...,iPhone 5s,iPhone 6,iPhone 6 Plus,iPhone 6S,iPhone 6S Plus,iPhone 7,iPhone 7 Plus,iPhone 8,iPhone 8 Plus,iPhone SE,iPhone X,nan
person,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1199.000000,NaN,NaN
00091926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1447.0,1639.243902,1733.479167,2000.764706,2442.555556,3121.222222,NaN,3634.0,1065.666667,5919.0,NaN
00091a7a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1839.0,NaN,1679.000000,NaN,NaN,NaN,NaN,NaN,1199.000000,NaN,NaN
000ba417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1139.0,NaN,NaN,NaN,NaN,2879.000000,NaN,NaN,NaN,NaN,NaN,NaN
000c79fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2469.000000,NaN,NaN,NaN,NaN,NaN,NaN


Vemos que hay ciertos usuarios para los cuales no tenemos información sobre el precio de los modelos que vieron. Para estos vamos a llenar todos sus valores con nulos (o ceros,  dado que en este caso luego los nulos los transformamos en ceros). Luego devolvemos los valores en forma de diccionario para el feature hashing.

In [57]:
d = grouped.fillna(0).to_dict('records')

Voy a trabajar con tantos 'features' como modelos haya. Voy a obtener un aproximado de la siguiente manera.

In [58]:
h = FeatureHasher(n_features=10)
f = h.transform(d)
features = pd.DataFrame(data=f.toarray()).add_prefix('precios_por_modelo_')
features.index = grouped.index

In [59]:
assert(features.shape[0] == df_features.shape[0])
features.head()

,precios_por_modelo_0,precios_por_modelo_1,precios_por_modelo_2,precios_por_modelo_3,precios_por_modelo_4,precios_por_modelo_5,precios_por_modelo_6,precios_por_modelo_7,precios_por_modelo_8,precios_por_modelo_9
person,,,,,,,,,,
0008ed71,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-759.000000,-659.000000,-1199.000000
00091926,-3974.984127,-519.000000,-1135.0,-529.0,379.000000,-3447.764706,497.666667,-9797.479167,-2720.888889,494.291521
00091a7a,0.000000,0.000000,0.0,0.0,0.000000,-1839.000000,0.000000,-1679.000000,0.000000,-1199.000000
000ba417,-3186.333333,-836.142857,-449.0,409.0,1035.636364,0.000000,-1019.142857,-325.000000,-825.500000,-434.360119
000c79fe,-2469.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
df_features = df_features.merge(features, how='inner', on='person')
assert(df_features.shape[0] == df['person'].unique().shape[0])

### precios por fecha

In [61]:
df_merged = df.merge(precios, how='left', on='sku')

Paso los timestamps a strings para poder hashearlos.

In [62]:
df_merged['timestamp'] = df_merged['timestamp'].astype('str')

In [63]:
%%time
grouped = df_merged.groupby(['person', 'timestamp']).agg({'price_current': 'mean'}).fillna(0)
grouped = grouped.reset_index(level=1).groupby('person').apply(lambda x: dict(x.to_dict(orient='split')['data']))

CPU times: user 3min 29s, sys: 764 ms, total: 3min 29s
Wall time: 3min 30s


In [64]:
h = FeatureHasher(n_features=100)
f = h.transform(d)
features = pd.DataFrame(data=f.toarray()).add_prefix('precios_por_fecha_')
features.index = grouped.index

In [65]:
assert(features.shape[0] == df_features.shape[0])
features.head()

,precios_por_fecha_0,precios_por_fecha_1,precios_por_fecha_2,precios_por_fecha_3,precios_por_fecha_4,precios_por_fecha_5,precios_por_fecha_6,precios_por_fecha_7,precios_por_fecha_8,precios_por_fecha_9,precios_por_fecha_10,precios_por_fecha_11,...,precios_por_fecha_88,precios_por_fecha_89,precios_por_fecha_90,precios_por_fecha_91,precios_por_fecha_92,precios_por_fecha_93,precios_por_fecha_94,precios_por_fecha_95,precios_por_fecha_96,precios_por_fecha_97,precios_por_fecha_98,precios_por_fecha_99
person,,,,,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-659.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
00091926,-2759.0,0.0,0.0,0.0,0.0,0.0,-928.0,0.0,-3648.222222,0.0,0.0,0.0,...,0.0,882.714286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1733.479167,0.0,0.0
00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1679.000000,0.0,0.0
000ba417,464.0,0.0,0.0,0.0,643.0,0.0,0.0,0.0,-536.500000,259.0,0.0,0.0,...,0.0,0.000000,0.0,-389.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-469.0
000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [66]:
df_features = df_features.merge(features, how='inner', on='person')
assert(df_features.shape[0] == df['person'].unique().shape[0])

In [67]:
df_features.head()

,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,precios_por_fecha_88,precios_por_fecha_89,precios_por_fecha_90,precios_por_fecha_91,precios_por_fecha_92,precios_por_fecha_93,precios_por_fecha_94,precios_por_fecha_95,precios_por_fecha_96,precios_por_fecha_97,precios_por_fecha_98,precios_por_fecha_99
person,,,,,,,,,,,,,,,,,,,,,,,,,
4886f805,640.0,360.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ad93850f,640.0,360.0,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
0297fc1e,640.0,360.0,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,-1879.0,0.0,0.0,-1691.192982,0.0,0.0
2d681dd8,768.0,1360.0,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
cccea85e,864.0,1536.0,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,...,0.0,2104.666667,619.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


## diferencia de precios

In [68]:
df_merged['price_difference'] = df_merged['price_current']/df_merged['price_new']

### diferencia de precios por modelo

### diferencia de precios por fecha

In [70]:
%%time
grouped = df_merged.groupby(['person', 'timestamp']).agg({'price_difference': 'mean'}).fillna(0)
grouped = grouped.reset_index(level=1).groupby('person').apply(lambda x: dict(x.to_dict(orient='split')['data']))

CPU times: user 3min 15s, sys: 564 ms, total: 3min 16s
Wall time: 3min 16s
